In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


# Self-Ask

This prompting method was introduced in the October 2022 paper titled "[Measuring and Narrowing the Compositionality Gap in Language Models](https://arxiv.org/pdf/2210.03350.pdf)" by Ofir Press, et. al.

### 🧠 **Compositional Reasoning in Language Models**

- Focuses on solving complex problems by piecing together answers to smaller sub-problems.

### 📏 **Introducing the 'Compositionality Gap'**

- A new metric to evaluate how models solve sub-problems but fail to integrate these solutions into a comprehensive answer.

### 📈 **Findings on Model Performance**

- Larger models like GPT-3 improve more on single-hop (simple) questions than multi-hop (complex) ones.

- This indicates the 'compositionality gap' doesn't shrink with bigger models.

### 🔍 **Challenge with Bigger Models**

- Despite remembering and recalling more facts, these models struggle with integrating these facts for complex problem-solving.

# 🤖 **Self-Ask Prompting Technique**

### 🔍 **How it Works**

1. **Generate Follow-Up Questions**: Instead of directly answering, the model first creates related, simpler questions.

2. **Answer the Follow-Ups**: The model then resolves these self-generated questions.

3. **Answer the Original Question**: Finally, it combines the answers to respond to the initial, complex question.

<figure>
  <img src="https://ar5iv.labs.arxiv.org/html/2210.03350/assets/x5.png" alt="Image Description" style="width:80%" style="height:40%">
  <figcaption>Direct prompting (Brown et al., 2020) compared to chain of thought and self-ask method on a question from Bamboogle. Text with a white background is the prompt, text with a green background is the LM output, and underlined text is the inference-time question</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2210.03350">Image Source</a>
</figure>


### 🌟 **Benefits of Self-Ask Technique**

- Promotes explicit reasoning by breaking down complex questions.

- Structured approach suitable for integrating with a search engine for improved accuracy.

- It's like the model having a conversation with itself to better tackle a problem.

### ✅ **Impact**

- Simplifies complex problems into manageable parts for a comprehensive final answer.



# 🔍**Understanding the Self-Ask Pseudo-Code**

You can find the original implementation of Self-Ask from the [paper's GitHub repo](https://github.com/ofirpress/self-ask/blob/main/self-ask_plus_search-engine_demo.ipynb).


### 🔑 **Key Components of the Code**

1. **`self_ask` Function**: The core function that handles the self-ask process.
   - Starts with an initial question.
   - Continuously generates and answers follow-up questions using an external source.
   - Ends with a final answer after resolving all follow-ups.

2. **Helper Functions**:
   - `frame_question`: Prepares the initial question for processing.
   - `call_gpt`: Simulates a call to the GPT model with the current prompt.
   - `get_last_line`: Extracts the last line of the text, often where follow-up questions or answers are found.
   - `extract_question`: Pulls out a follow-up question from the text.
   - `get_answer`: Represents fetching an answer from an external source.

### 🔄 **Workflow of the Algorithm**

- Begin with framing the initial question.
- Enter a loop where:
  - If a follow-up question is detected, an external answer is sought.
  - This external answer is added to the current prompt, and the model is called again.
  - If there's no external answer, it proceeds to seek the final answer.
- The process repeats until a final answer is formulated.



```python

def self_ask(question: str) -> str:
    cur_prompt = frame_question(question)  
    ret_text = call_gpt(cur_prompt)

    while "Follow up:" in get_last_line(ret_text):
        extracted_question = extract_question(ret_text)

        external_answer = get_answer(extracted_question)

        if external_answer:
            cur_prompt += "Intermediate answer: " + external_answer
            ret_text = call_gpt(cur_prompt)
        else:
            cur_prompt += "Intermediate answer: "
            ret_text = call_gpt(cur_prompt + " Follow up: " + "Final answer: ")

    if "Final answer:" not in ret_text:
        cur_prompt += "Final answer: "
        ret_text = call_gpt(cur_prompt)

    return cur_prompt + ret_text

def frame_question(question: str) -> str:
    return question

def call_gpt(prompt: str) -> str:
    return ""

def get_last_line(text: str) -> str:
    return text.split('\n')[-1] if '\n' in text else text

def extract_question(text: str) -> str:

def get_answer(question: str) -> str:
    return ""
```

# LangChain makes Self-Ask easy

In [ ]:
pip install langchain langchain-openai langchain-community duckduckgo-search

In [8]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun

llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

search = DuckDuckGoSearchRun()

In [9]:
from langchain.agents import Tool

tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [ ]:
pip install langchainhub

In [12]:
prompt = hub.pull("hwchase17/self-ask-with-search")

In [13]:
print(prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [14]:
from langchain.prompts import PromptTemplate

deep_learning_template = """
Question: Which architecture is more suited for image classification, LSTM or CNN?
Are follow-up questions needed here: Yes.
Follow-up 1: What is the primary application of LSTM?
Intermediate Answer: LSTM (Long Short-Term Memory) is primarily used for sequential data and time series analysis.
Follow-up 2: What is the primary application of CNN?
Intermediate Answer: CNN (Convolutional Neural Network) is primarily used for image and video recognition.
Final Answer: CNN is more suited for image classification.

Question: Which optimizer converges faster, SGD or Adam?
Are follow-up questions needed here: Yes.
Follow-up 1: What is the main advantage of SGD (Stochastic Gradient Descent)?
Intermediate Answer: SGD is simple and easy to implement.
Follow-up 2: What is the main advantage of Adam optimizer?
Intermediate Answer: Adam combines the benefits of both AdaGrad and RMSProp and usually converges faster than SGD.
Final Answer: Adam converges faster.

Question: For natural language processing tasks, which dataset is larger, MNIST or WikiText-103?
Are follow-up questions needed here: Yes.
Follow-up 1: What kind of data does the MNIST dataset contain?
Intermediate Answer: MNIST contains handwritten digits and is used for image classification.
Follow-up 2: What kind of data does the WikiText-103 dataset contain?
Intermediate Answer: WikiText-103 is a large language modeling dataset containing text from Wikipedia articles.
Final Answer: WikiText-103 is larger and is used for natural language processing tasks.

Question: {input}
Are followup questions needed here:{agent_scratchpad}
"""

deep_learning_prompt = PromptTemplate.from_template(deep_learning_template)

# 🔧 **Binding a Stop Sequence in LLM Generation**

- The concept involves using a specific string (in this case, `"\nIntermediate answer:"`) as a signal for the language model (LLM) to halt text generation.

### 🎯 **Purpose in Self-Ask with Search Agent**

- Facilitates a structured dialogue between the LLM and the search agent.

- The LLM stops after providing an intermediate answer, allowing the search agent to process this information and pose a follow-up question.

### 🛑 **How the Stop Sequence Works**

- Acts as a delimiter in the LLM's output.

- Once this sequence is generated, it indicates the end of the current response phase and triggers subsequent processing or querying steps.

%%html
<figure>
  <img src="https://ar5iv.labs.arxiv.org/html/2210.03350/assets/x9.png" alt="Image Description" style="width:100%">
  <figcaption>Self-ask + Search Engine: prompt on white background, LM-generated text in green. Start by using a few-shot prompt and appending the test-time question (underlined) to it. The LM then generates a follow-up question which inputs to an internet search engine. The response is inserted back into the rest of the prompt to let the LM generate the next follow-up question. This process then repeats until the LM decides to output the final answer.</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2210.03350">Image Source</a>
</figure>


In [15]:
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import SelfAskOutputParser
from langchain.agents.format_scratchpad import format_log_to_str

llm_with_stop = llm.bind(stop=["Intermediate Answer:"])

agent = {
    "input": lambda x: x["input"],
    # Use some custom observation_prefix/llm_prefix for formatting
    "agent_scratchpad": lambda x: format_log_to_str(
        x['intermediate_steps'],
        observation_prefix="Intermediate Answer: ",
        llm_prefix="",
    ),
} | prompt | llm_with_stop | SelfAskOutputParser()

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               handle_parsing_errors=True
                               )

In [16]:
agent_executor.invoke({"input": "What is the RWKV architecture for LLMs?"})



> Entering new AgentExecutor chain...
Yes.
Follow up: What does RWKV stand for in the context of LLMs?
Intermediate answer: RWKV stands for Random Window Key Value.
Follow up: What is the purpose of the RWKV architecture?
Intermediate answer: The RWKV architecture is designed to improve the efficiency and effectiveness of language models by introducing a mechanism that allows for more dynamic attention across different parts of the input text, enhancing the model's ability to understand and generate human-like text.
So the final answer is: The RWKV architecture for LLMs (Large Language Models) stands for Random Window Key Value, aimed at improving model efficiency and text understanding through a dynamic attention mechanism.

> Finished chain.


{'input': 'What is the RWKV architecture for LLMs?',
 'output': 'The RWKV architecture for LLMs (Large Language Models) stands for Random Window Key Value, aimed at improving model efficiency and text understanding through a dynamic attention mechanism.'}

In [17]:
agent_executor.invoke({"input": "How is Direct Preference Optimization different than traditional RLHF?"})



> Entering new AgentExecutor chain...
Yes.
Follow up: What is Direct Preference Optimization?
Intermediate answer: Direct Preference Optimization (DPO) is a method in machine learning where the model is trained directly on preferences between different outputs or actions, rather than on explicit rewards or labels. This approach is particularly useful in scenarios where specifying a reward function is difficult or infeasible, and it relies on comparisons made by humans or other agents to guide the learning process.
Follow up: What is traditional RLHF?
Intermediate answer: Traditional Reinforcement Learning from Human Feedback (RLHF) involves training a model using a combination of reinforcement learning (RL) and human feedback. In this approach, human feedback is used to shape or augment the reward signal that guides the learning process of the model. This can involve techniques like reward modeling, where human feedback is used to create a model of the reward function that the RL alg

{'input': 'How is Direct Preference Optimization different than traditional RLHF?',
 'output': 'Direct Preference Optimization (DPO) differs from traditional RLHF in that DPO trains models directly on human preferences between outputs without the need for a reward model, whereas traditional RLHF uses human feedback to shape or create a reward function that guides the reinforcement learning process.'}

# New way

In [18]:
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_openai import ChatOpenAI
from langchain import hub

llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

prompt = hub.pull("hwchase17/self-ask-with-search")

tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

agent = create_self_ask_with_search_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               handle_parsing_errors=True)

agent_executor.invoke({"input": "Who is Hippie Sabatoge and where are they from?"})



> Entering new AgentExecutor chain...
Yes.
Follow up: What is Hippie Sabotage?Hippie Sabotage frequently performs "Trust Nobody" during live shows, allowing fans to immerse themselves in the emotions evoked by the song in a live setting. About The Author. Bayram Bösch. Bayram Bösch is a frequent contributor to OldTimeMusic. the room is spinnin but I see straight....new track with our friend izzy bizu is out now ️ http://tr.ee/boyproblems music video by michael paredesassistant:... listen to "assassin in the window": http://tr.ee/aitwsee you at the ENTER THE UNKNOWN tour 🛸🏛️ get tickets: http://tr.ee/entertheunknownsong by jeff & kevi... our album 'trailblazer' is out now. check out music, merch, and more at http://linktr.ee/hippiesabotage 🌴Lyrics:I'm smoking homegrownI grow my potI'm in m... The EDM duo Hippie Sabotage found fame with a genre-blurring stoner sound characterized by lax vocals, downtempo beats, and a jazzy mix of organic instrumentation and electronic flourishes. • Hip

{'input': 'Who is Hippie Sabatoge and where are they from?',
 'output': 'Hippie Sabotage is an EDM duo from California.'}